In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [35]:
# 获得mnist数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../Resources/mnistdata", one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print(trainimg.shape, trainlabel.shape)
print(testimg.shape, testlabel.shape)

Extracting ../Resources/mnistdata\train-images-idx3-ubyte.gz
Extracting ../Resources/mnistdata\train-labels-idx1-ubyte.gz
Extracting ../Resources/mnistdata\t10k-images-idx3-ubyte.gz
Extracting ../Resources/mnistdata\t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


In [36]:
# 定义cnn
n_input = 784
n_output = 10

weights = {
    # 5*5 conv  1 input  32 output
    "wc1" : tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5*5 conv  32 input  64 output
    "wc2" : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected  7*7*64 input  1024 output
    # 原始28*28经过两个最大池化层 大小变为7*7 通道64
    "wd1" : tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 input 10 output
    "out" : tf.Variable(tf.random_normal([1024, n_output]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([32])),
    "bc2" : tf.Variable(tf.random_normal([64])),
    "bd1" : tf.Variable(tf.random_normal([1024])),
    "out" : tf.Variable(tf.random_normal([n_output]))
}


def conv2d(x, W, b, strides=1):
    conv = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    conv = tf.nn.bias_add(conv, b)
    return tf.nn.relu(conv)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def conv_net(x, weights, biases, dropout):
    # mninst 数据类型是 1-D vector，共784features
    # 需进行reshape
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]  灰度图像通道数为1
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # conv layer1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # max pooling
    conv1 = maxpool2d(conv1)
    
    # conv layer2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # max pooling
    conv2 = maxpool2d(conv2)
    
    # fully connected layer
    # reshape get_shape()返回的是一个元组 需要转化为list
    fc1 = tf.reshape(conv2, shape=[-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    return tf.add(tf.matmul(fc1, weights['out']), biases['out'])

In [37]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keep_prob = tf.placeholder(tf.float32)
# params
learning_rate = 0.001
dropout = 0.75
training_epochs = 2000
batch_size = 128
display_step = 20

pred = tf.nn.softmax(conv_net(x, weights, biases, keep_prob))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [38]:
# start training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        n_batch = int(mnist.train.num_examples / batch_size)
        for i in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds = {x: batch_xs, y:batch_ys, keep_prob: dropout}
            sess.run(optimizer, feed_dict=feeds)
            feeds = {x: batch_xs, y:batch_ys, keep_prob: 1.0}
            avg_cost += sess.run(cost, feed_dict=feeds)
        avg_cost /= n_batch
        
        if epoch % display_step == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
            train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y:batch_ys, keep_prob: 1.0})
            print("TRAIN ACCURACY: %.3f" % (train_acc))
            # 由于显卡显存限制 测试集无法一次加载 需要分批计算
            test_acc = 0
            n_test_batch = int(mnist.test.num_examples / batch_size)
            for i in range(n_test_batch):
                batch_xs, batch_ys = mnist.test.next_batch(batch_size)
                test_acc += sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.0})
            print ("TEST ACCURACY: %.3f" % (test_acc / n_test_batch))
            

Epoch: 000/200 cost: 2.364759349
TRAIN ACCURACY: 0.133
TEST ACCURACY: 0.098
Epoch: 005/200 cost: 2.364431784
TRAIN ACCURACY: 0.133
TEST ACCURACY: 0.098
Epoch: 010/200 cost: 2.364541049
TRAIN ACCURACY: 0.070
TEST ACCURACY: 0.099
Epoch: 015/200 cost: 2.364668526
TRAIN ACCURACY: 0.102
TEST ACCURACY: 0.098
Epoch: 020/200 cost: 2.364613893
TRAIN ACCURACY: 0.102
TEST ACCURACY: 0.098
Epoch: 025/200 cost: 2.364723159
TRAIN ACCURACY: 0.117
TEST ACCURACY: 0.097
Epoch: 030/200 cost: 2.365142011
TRAIN ACCURACY: 0.109
TEST ACCURACY: 0.098
Epoch: 035/200 cost: 2.364632104
TRAIN ACCURACY: 0.109
TEST ACCURACY: 0.099
Epoch: 040/200 cost: 2.364413573
TRAIN ACCURACY: 0.102
TEST ACCURACY: 0.098
Epoch: 045/200 cost: 2.365269488
TRAIN ACCURACY: 0.078
TEST ACCURACY: 0.098
Epoch: 050/200 cost: 2.365251277
TRAIN ACCURACY: 0.094
TEST ACCURACY: 0.098
Epoch: 055/200 cost: 2.363830822
TRAIN ACCURACY: 0.094
TEST ACCURACY: 0.099
Epoch: 060/200 cost: 2.365123800
TRAIN ACCURACY: 0.109
TEST ACCURACY: 0.097


KeyboardInterrupt: 